In [ ]:
import pandas as pd

#Questão 1

###Leitura de Arquivos


In [ ]:
df_entregas = pd.read_csv('entregas_tab.txt', sep = '|', skiprows = 5, encoding = 'utf-8', dtype = {1:'str'})

In [ ]:
df_produtos = pd.read_csv('prods_tab.csv', sep = ',', skiprows = 0, encoding = 'latin-1', dtype = {0:'str'})

###Renomeando Colunas

In [ ]:
df_entregas.columns = [var.replace('.',' ').strip().lower().replace(' ','_') for var in df_entregas.columns]

In [ ]:
df_entregas.columns

Index(['unnamed:_0', 'sku', 'dt_compra', 'dt_chegada', 'sktd', 'unnamed:_5'], dtype='object')

In [ ]:
df_produtos.columns = ['fornecedor', 'prod_desc', 'sku', 'grp_merc_3', 'cod_marc']

###Eliminando Colunas


In [ ]:
df_entregas = df_entregas[['sku','dt_compra', 'dt_chegada']].copy().dropna(axis = 0, subset = ['dt_compra'])

In [ ]:
df_entregas

,sku,dt_compra,dt_chegada
1,100312,2017-12-02,2018-01-05
2,89721,2017-12-02,2018-01-05
3,192063,2017-12-05,2018-01-04
4,182285,2017-12-05,2018-01-07
5,32934,2017-12-10,2018-01-05
6,43495,2017-12-02,2018-01-12
7,4342,2017-12-02,2018-01-10
8,3124,2017-12-04,2018-01-04
9,43495,2017-12-07,2018-01-05


In [ ]:
df_produtos = df_produtos[['fornecedor','sku']].copy().dropna(axis = 0, subset = ['fornecedor']).drop([5, 8], axis = 0)

In [ ]:
df_produtos

,fornecedor,sku
1,302100012,100312-
2,302100012,89 721 ?
3,302100012,?1920-- 63
6,302100012,18228 5
10,9009101002,32 934
11,9030121093,4??349 5
12,320621093,4342I
13,320621093,3 1---24-
14,9030121093,1 92501


###Tratar e Alterar tipos de Colunas

In [ ]:
df_produtos['fornecedor'] = df_produtos['fornecedor']

In [ ]:
df_produtos['sku'] = df_produtos['sku'].str.strip().str.replace('-','').str.replace('?','').str.replace(' ','').str.replace('I','')

In [ ]:
df_entregas['dt_chegada'] = df_entregas['dt_chegada'].str.strip()

In [ ]:
df_entregas['dt_compra'] = df_entregas['dt_compra'].str.strip()

In [ ]:
df_entregas['sku'] = df_entregas['sku'].str.strip()

In [ ]:
df_produtos.dtypes

fornecedor    object
sku           object
dtype: object

In [ ]:
df_entregas.dtypes

sku           object
dt_compra     object
dt_chegada    object
dtype: object

In [ ]:
df_entregas['dt_compra'] = pd.to_datetime(df_entregas['dt_compra'], format='%Y.%m.%d')

In [ ]:
df_entregas['dt_chegada'] = pd.to_datetime(df_entregas['dt_chegada'], format='%Y.%m.%d')

###Manipulação de Dados

In [ ]:
df_produtos

,fornecedor,sku
1,302100012,100312
2,302100012,89721
3,302100012,192063
6,302100012,182285
10,9009101002,32934
11,9030121093,43495
12,320621093,4342
13,320621093,3124
14,9030121093,192501


In [ ]:
df_entregas

,sku,dt_compra,dt_chegada
1,100312,2017-12-02,2018-01-05
2,89721,2017-12-02,2018-01-05
3,192063,2017-12-05,2018-01-04
4,182285,2017-12-05,2018-01-07
5,32934,2017-12-10,2018-01-05
6,43495,2017-12-02,2018-01-12
7,4342,2017-12-02,2018-01-10
8,3124,2017-12-04,2018-01-04
9,43495,2017-12-07,2018-01-05


In [ ]:
df = pd.merge(df_entregas,df_produtos, on = ['sku'], how = 'left')

In [ ]:
df['lt'] = df['dt_chegada'] - df['dt_compra']

In [ ]:
df['lt'] = df['lt'].astype('timedelta64[D]').astype(int)

In [ ]:
df = df[['fornecedor','lt']].copy()

###Resultado

In [ ]:
df.groupby('fornecedor').mean()

,lt
fornecedor,
302100012,32.75
320621093,35.00
9009101002,26.00
9030121093,35.00


In [ ]:
df.describe()

,lt
count,9.000000
mean,33.000000
std,4.743416
min,26.000000
25%,30.000000
50%,33.000000
75%,34.000000
max,41.000000


#Questão 2

###Leitura de Arquivos

In [ ]:
df_bonus = pd.read_csv('bonus.txt', sep = ';', skiprows = 0, encoding = 'latin-1', dtype = {0:'str'})

In [ ]:
df_clientes = pd.read_csv('clientes.txt', sep = ';', skiprows = 0, encoding = 'utf-8', dtype = {0:'str'}, names=['cliente', 'dt_cadastro','codigo','ddd','estado_civl','ens_medio','graduacao','num_filhos','cnh','cartao_da_loja','sorteios_ganhos','renda','comprov_renda','cartao','num_reclamacoes','sexo','num'])

###Unindo tabelas e criando colunas

In [ ]:
df_total = pd.merge(df_clientes,df_bonus, on = ['cliente'], how = 'left')

In [ ]:
df_total['pontos_totais'] = df_total['saldo'] + df_total['historico']

In [ ]:
def pedra(x):
  if x <= 6000:
    return 'AZUL'
  elif x <= 10000:
    return 'PRATA'
  elif x <= 20000:
    return 'OURO'
  elif x <= 50000:
    return 'DIAMANTE'
  elif x > 50000:
    return 'DIAMANTE+'

In [ ]:
df_total['classificacao'] = [pedra(x) for x in df_total['pontos_totais']]

###Resultado


In [ ]:
df_total

,cliente,dt_cadastro,codigo,ddd,estado_civl,ens_medio,graduacao,num_filhos,cnh,cartao_da_loja,sorteios_ganhos,renda,comprov_renda,cartao,num_reclamacoes,sexo,num,saldo,historico,soma_pontos,uso_pontos,expirado,avista,soma_avista,aprazo,soma_aprazo,soma_pontualidade,pontos_totais,classificacao
0,1,23/10/1999,61438761231,61,Casado(a),S,S,2,S,N,0,"2.001,00-4.000,00",Não Possui,Visa,149,F,N04,2095,2095,0,0,2095,0,0,10,0.0,0,4190,AZUL
1,2,23/10/1999,7539570251,66,Casado(a),N,N,3,N,N,21,"Até 1.000,00",Não Possui,Nenhum,8,M,N05,1283,1283,0,0,1283,2,0,4,0.0,0,2566,AZUL
2,3,23/10/1999,5429217161,66,Solteiro(a),S,S,2,S,N,5,"2.001,00-4.000,00",Compr. de Atividade,Mastercard+Visa,91,M,N12,1000,1000,0,0,1000,0,0,6,0.0,0,2000,AZUL
3,4,23/10/1999,1761017149,83,Casado(a),S,S,1,S,S,0,"1.001,00-2.000,00",Ct-chq/IR (NAO USAR),Nenhum,378,M,N12,1000,1000,0,0,1000,0,0,3,0.0,0,2000,AZUL
4,5,23/10/1999,4321391204,70,Casado(a),S,S,2,N,S,0,"1.001,00-2.000,00",Não Possui,Nenhum,19,F,N04,3235,30235,13491,9000,0,45,292,168,7684.7,1138,33470,DIAMANTE
5,6,23/10/1999,13024171220,73,Casado(a),N,N,0,S,N,0,"Até 1.000,00",Não Possui,Nenhum,20,M,N01,3434,27434,1500,0,2734,28,0,14,0.0,0,30868,DIAMANTE
6,7,23/10/1999,110000,81,Separado(a),N,N,2,N,N,0,NaN,Não Possui,Nenhum,30,F,N05,3673,15673,12024,12000,485,27,304,48,6487.0,598,19346,OURO
7,8,23/10/1999,14019711215,63,Casado(a),N,N,2,N,S,0,"Até 1.000,00",Não Possui,Nenhum,29,M,N07,2467,8467,0,0,2311,4,0,25,0.0,0,10934,OURO
8,9,23/10/1999,5384501115,70,Casado(a),N,S,0,N,N,9,"2.001,00-4.000,00",Não Possui,Nenhum,32,M,N13,2924,29924,6319,6000,0,5,0,73,3468.0,651,32848,DIAMANTE
9,10,23/10/1999,13534871187,63,Casado(a),S,S,2,S,N,15,"2.001,00-4.000,00",Ct-chq/IR (NAO USAR),Outros/Cartões Lojas,0,M,N04,1000,1000,0,0,1000,0,0,10,0.0,0,2000,AZUL


In [ ]:
df_total.to_csv('DADOS.txt', sep = '|', index = False)

#Questão 3

In [ ]:
!pip install plotly --upgrade
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

###Leitura de Arquivos

In [ ]:
df_realizado = pd.read_excel('dados.xlsx', sheet_name = 0, dtype = {0:'str'})

In [ ]:
df_orcado = pd.read_excel('dados.xlsx', sheet_name = 1, dtype = {0:'str'})

###Renomeando colunas


In [ ]:
df_orcado

,Unnamed: 0,Mes_1,Mes_2,Mes_3,Mes_4,Mes_5,Mes_6,Mes_7,Mes_8,Mes_9,Mes_10,Mes_11,Mes_12
0,NaN,janeiro,fevereiro,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro
1,orcado,330,290,230,321,283,291,193,259,289,230,434,421


In [ ]:
df_realizado.columns

Index(['Unnamed: 0', 'Mes_1', 'Mes_2', 'Mes_3', 'Mes_4', 'Mes_5', 'Mes_6',
       'Mes_7', 'Mes_8', 'Mes_9', 'Mes_10', 'Mes_11', 'Mes_12'],
      dtype='object')

###Elimando linhas e colunas

In [ ]:
df_orcado = df_orcado.drop('Unnamed: 0', axis = 1)

In [ ]:
df_realizado = df_realizado.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df_orcado = df_orcado.T

In [ ]:
df_realizado = df_realizado.T

In [ ]:
df_orcado.columns = ['mes','orcado']

In [ ]:
df_realizado.columns = ['mes','realizado']

In [ ]:
df_orcado['orcado'] = df_orcado['orcado'].astype(int)

In [ ]:
df_realizado['realizado'] = df_realizado['realizado'].astype(int)

###Unindo tabelas e criando colunas

In [ ]:
df_dados = pd.merge(df_orcado,df_realizado, on = ['mes'], left_index=True, right_index=True)

In [ ]:
df_dados['diferenca'] = df_dados['orcado'] - df_dados['realizado']

In [ ]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, Mes_1 to Mes_12
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   mes        12 non-null     object
 1   orcado     12 non-null     int64 
 2   realizado  12 non-null     int64 
 3   diferenca  12 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 800.0+ bytes


In [ ]:
graf_orcamento = px.bar(df_dados, 
             x='mes', 
             y=['orcado', 'realizado'], 
             barmode='overlay',
             title='Gráfico Orçamento',
             labels={"mes": "Mês", "value": "$"})
graf_orcamento.show()

In [ ]:
df_dados = df_dados.T.copy()
df_dados.to_csv('dados_saidas.csv')